# TECHNICAL REPORT: Multi-Trip VRP (2-Vehicle Fleet)

## 1. Scenario Definition

In this version, a realistic operation is modeled where the fleet is limited, but time is not. 

**Parameters:**
* **Capacity:** 80 kg (Standard).
* **Physical Fleet:** 2 Vehicles.
* **Operation:** Vehicles can return to the depot, reload, and perform a second trip.

**Solution Logic:**
The Genetic Algorithm will calculate the necessary trips (logical routes) to meet all demand. Subsequently, an assignment algorithm will distribute these trips among the 2 available vehicles to balance the workload.

In [ ]:
import random
import math
import matplotlib.pyplot as plt

# Data
depot = [20, 120]
clientes = [
    [35, 115], [50, 140], [70, 100], [40, 80], [25, 60]
]
pesos = [1.2, 3.8, 7.5, 0.9, 15.4, 12.1, 4.3, 19.7, 8.6, 2.5]
pedidos = [
    [(3, 2), (1, 3)],
    [(2, 6)],
    [(7, 4), (5, 2)],
    [(3, 8)],
    [(6, 5), (9, 2)]
]

capacidad = 80
n_vehiculos_fisicos = 2

In [ ]:
# Demand Calculation
demandas = []
for pedido in pedidos:
    total = 0
    for item, cant in pedido:
        total += pesos[item - 1] * cant
    demandas.append(total)

clientes_validos = []
demandas_validas = []
for i in range(len(clientes)):
    if demandas[i] <= capacidad:
        clientes_validos.append(clientes[i])
        demandas_validas.append(demandas[i])

print(f"Total Demand to serve: {sum(demandas_validas)} kg")

In [ ]:
# --- GA Logic (Identical to base version) ---
# The GA seeks to minimize the total distance of necessary trips.
# We will then assign those trips to the cars.

def distancia(a, b):
    return math.sqrt((a[0] - b[0])**2 + (a[1] - b[1])**2)

def dividir_en_rutas(ruta):
    rutas = []
    carga = 0
    actual = []
    for c in ruta:
        if carga + demandas_validas[c] <= capacidad:
            actual.append(c)
            carga += demandas_validas[c]
        else:
            rutas.append(actual)
            actual = [c]
            carga = demandas_validas[c]
    if actual:
        rutas.append(actual)
    return rutas

def distancia_total(ruta):
    rutas = dividir_en_rutas(ruta)
    total = 0
    for r in rutas:
        pos = depot
        for c in r:
            total += distancia(pos, clientes_validos[c])
            pos = clientes_validos[c]
        total += distancia(pos, depot)
    return total

def fitness(ruta):
    return distancia_total(ruta)

def crear_poblacion(n, n_clientes):
    poblacion = []
    base = list(range(n_clientes))
    for _ in range(n):
        r = base[:]
        random.shuffle(r)
        poblacion.append(r)
    return poblacion

def seleccion(poblacion, fitnesses):
    i1, i2 = random.sample(range(len(poblacion)), 2)
    return poblacion[i1][:] if fitnesses[i1] < fitnesses[i2] else poblacion[i2][:]

def cruce(p1, p2):
    a, b = sorted(random.sample(range(len(p1)), 2))
    hijo = [-1] * len(p1)
    hijo[a:b] = p1[a:b]
    pos = b
    for x in p2:
        if x not in hijo:
            if pos >= len(p1): pos = 0
            hijo[pos] = x
            pos += 1
    return hijo

def mutacion(ruta, prob=0.1):
    if random.random() < prob:
        i, j = random.sample(range(len(ruta)), 2)
        ruta[i], ruta[j] = ruta[j], ruta[i]

def algoritmo_genetico(n_generaciones=200, tam_pob=50):
    poblacion = crear_poblacion(tam_pob, len(clientes_validos))
    fitnesses = [fitness(r) for r in poblacion]

    for _ in range(n_generaciones):
        nueva = []
        for _ in range(tam_pob):
            p1 = seleccion(poblacion, fitnesses)
            p2 = seleccion(poblacion, fitnesses)
            hijo = cruce(p1, p2)
            mutacion(hijo, 0.1)
            nueva.append(hijo)
        poblacion = nueva
        fitnesses = [fitness(r) for r in poblacion]

    mejor = min(range(tam_pob), key=lambda i: fitnesses[i])
    return poblacion[mejor], fitnesses[mejor]

In [ ]:
# Execute GA to get necessary logical routes
mejor_ruta, mejor_valor = algoritmo_genetico()
viajes_necesarios = dividir_en_rutas(mejor_ruta)

print(f"Total trips necessary: {len(viajes_necesarios)}")

# --- ASSIGNMENT OF TRIPS TO PHYSICAL VEHICLES ---
# Simple 'Round Robin' assignment to balance
vehiculos_asignados = {0: [], 1: []} # Vehicle 0 and Vehicle 1

for i, viaje in enumerate(viajes_necesarios):
    id_vehiculo = i % n_vehiculos_fisicos
    vehiculos_asignados[id_vehiculo].append(viaje)

print("\n--- Fleet Planning ---")
for v_id, viajes in vehiculos_asignados.items():
    print(f"PHYSICAL VEHICLE {v_id + 1}:")
    if not viajes:
        print("  (No activity)")
    for idx, r in enumerate(viajes):
        carga = sum(demandas_validas[c] for c in r)
        print(f"  -> Shift {idx+1}: Customers {r} | Load: {round(carga,1)} kg")

In [ ]:
def plot_multitrip(vehiculos_asignados):
    plt.figure(figsize=(9, 9))
    
    # Depot
    plt.scatter(depot[0], depot[1], c='red', s=150, marker='s', label='Depot', zorder=10)
    
    # Customers
    for i, (x, y) in enumerate(clientes_validos):
        plt.scatter(x, y, c='blue', zorder=5)
        plt.text(x+1, y+1, f'C{i+1}', fontsize=9)

    colores_vehiculo = ['#1f77b4', '#ff7f0e'] # Blue and Orange
    estilos_linea = ['-', '--', ':'] # Solid, Dashed, Dotted (to distinguish shifts)
    
    for v_id, viajes in vehiculos_asignados.items():
        color = colores_vehiculo[v_id]
        
        for idx_viaje, r in enumerate(viajes):
            estilo = estilos_linea[idx_viaje % len(estilos_linea)]
            
            puntos = [depot] + [clientes_validos[c] for c in r] + [depot]
            xs = [p[0] for p in puntos]
            ys = [p[1] for p in puntos]
            
            label = f'V{v_id+1} - Shift {idx_viaje+1}'
            plt.plot(xs, ys, linestyle=estilo, color=color, linewidth=2, label=label, alpha=0.8)
            
            # Annotate load in the center of the route
            mid = len(xs) // 2
            plt.text(xs[mid], ys[mid], f'V{v_id+1}.{idx_viaje+1}', 
                     color=color, fontweight='bold', bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))

    plt.title("Multi-Trip Operation (2 Vehicles)")
    plt.xlabel("X")
    plt.ylabel("Y")
    plt.legend()
    plt.grid(True, linestyle='--', alpha=0.6)
    plt.show()

plot_multitrip(vehiculos_asignados)

## 5. Final Conclusions (Multi-Trip Scenario)

By allowing vehicles to make multiple trips (reload at depot), full demand is met under the original constraints (80 kg Capacity and 2 Vehicles):

1.  **Double Shift Operation:**
    * Demand analysis (227.1 kg) determines that **3 trips** in total are necessary.
    * With 2 vehicles available, the optimal solution implies an asymmetry in workload:
        * **Vehicle 1:** Performs **2 Shifts**. (Ex: A heavy trip with Customer 5, return, and a second light trip).
        * **Vehicle 2:** Performs **1 Shift**.

2.  **Asset Efficiency:**
    * This strategy avoids investment in a third truck or larger trucks.
    * The cost to pay is not monetary (investment), but temporal: total operation time extends because a vehicle must wait to return to the depot to start its second route.

3.  **Flexibility:**
    * The model demonstrates robustness. Although the 80 kg capacity is limiting (highly fragments orders), the 'return and reload' policy allows 100% service completion without leaving customers unattended.